In [ ]:
import time
import collections
import itertools
import heapq # Hint: Useful for priority queues

# --- Constants ---
PENDING = "PENDING"
RUNNING = "RUNNING"
COMPLETED = "COMPLETED"
FAILED = "FAILED"

# --- Job Representation ---
# Example: {'id': ..., 'command': ..., 'required_slots': ..., 'priority': ..., 'state': ...,
#           'submit_time': ..., 'start_time': ..., 'simulated_duration': ...,
#           'depends_on': set(), 'successors': set(), 'unmet_dependencies': int}

class DependencyScheduler:
    """
    A basic, in-memory, priority-based job scheduler simulating HPC resource
    management with job dependencies (DAG).
    Jobs with lower priority numbers run first, but only after dependencies are met.
    """

    def __init__(self, total_slots: int):
        """
        Initializes the scheduler.

        Args:
            total_slots: The total number of compute slots available.
        """
        if total_slots <= 0:
            raise ValueError("Total slots must be positive.")

        self.total_slots: int = total_slots
        self.free_slots: int = total_slots
        # Use a list as a min-heap (priority queue) for jobs ready to be considered
        # Store tuples: (priority, submit_time, job_id)
        # We store job_id now because the job_info dict will be updated elsewhere.
        self.pending_jobs_heap = []
        self.running_jobs = {} # {job_id: job_info}
        self.completed_jobs = {} # {job_id: job_info}
        self.job_id_counter = itertools.count(1)
        # Central dictionary to store all job info, including dependency details
        self.all_jobs = {} # {job_id: job_info}

        print(f"Scheduler initialized with {self.total_slots} slots.")

    def submit_job(self, command: str, required_slots: int, priority: int = 10, depends_on: list[int] | None = None) -> int:
        """
        Submits a new job to the scheduler with priority and dependencies.

        Args:
            command: The command to be executed (simulated).
            required_slots: The number of slots the job needs.
            priority: Job priority (lower number means higher priority). Defaults to 10.
            depends_on: A list of job IDs that this job depends on. Defaults to None (no dependencies).

        Returns:
            The unique ID assigned to the job.

        Raises:
            ValueError: If requirements are invalid or a dependency ID doesn't exist.
        """
        if required_slots <= 0:
            raise ValueError("Required slots must be positive.")
        if required_slots > self.total_slots:
            raise ValueError(f"Job requires {required_slots} slots, but scheduler only has {self.total_slots} total.")
        if not isinstance(priority, int):
             raise ValueError("Priority must be an integer.")

        depends_on_set = set(depends_on) if depends_on else set()
        unmet_dependency_count = 0

        # --- Candidate TODO: Validate dependencies and calculate initial unmet count ---
        # - Check if all job IDs in depends_on_set exist in self.all_jobs.
        # - Check if any dependency is already COMPLETED. If so, it doesn't count towards unmet.
        # - Raise ValueError for invalid/unknown dependencies.
        # - Initialize unmet_dependency_count based on non-completed dependencies.
        pass # Replace with validation logic

        job_id = next(self.job_id_counter)
        submit_time = time.time()
        job_info = {
            'id': job_id,
            'command': command,
            'required_slots': required_slots,
            'priority': priority,
            'state': PENDING,
            'submit_time': submit_time,
            'start_time': None,
            'simulated_duration': 5.0, # Example: fixed duration
            'depends_on': depends_on_set,
            'successors': set(), # Will be populated by jobs that depend on this one
            'unmet_dependencies': unmet_dependency_count # Calculated above
        }

        # Store the job centrally
        self.all_jobs[job_id] = job_info

        # --- Candidate TODO: Update successor lists for dependency jobs ---
        # For each job_id in depends_on_set, add the current job_id to its 'successors' set.
        pass # Replace with successor update logic

        # --- Candidate TODO: Add job to the pending heap *only if* it has no unmet dependencies ---
        # If unmet_dependencies == 0:
        #   heapq.heappush(self.pending_jobs_heap, (priority, submit_time, job_id))
        pass # Replace with heap push logic

        print(f"Job {job_id} ('{command}') submitted with priority {priority}, requiring {required_slots} slots, depends on {depends_on_set or '{}'}.")
        if unmet_dependency_count > 0:
            print(f"  Job {job_id} is waiting for {unmet_dependency_count} dependencies.")
        else:
             print(f"  Job {job_id} added to pending queue.")

        return job_id

    def _check_running_jobs(self):
        """
        Internal helper to check running jobs for completion.
        If a job completes, update the dependency counts of its successors.
        """
        current_time = time.time()
        completed_ids_this_cycle = []

        # Iterate safely
        for job_id in list(self.running_jobs.keys()):
            job = self.running_jobs[job_id]
            if job['start_time'] is not None and current_time >= job['start_time'] + job['simulated_duration']:
                # --- Candidate TODO: Handle job completion ---
                # 1. Update job state to COMPLETED.
                # 2. Release slots (increment self.free_slots).
                # 3. Move job from running_jobs to completed_jobs.
                # 4. Process dependencies:
                #    - Iterate through job['successors'].
                #    - For each successor_id:
                #        - Get the successor_job_info from self.all_jobs.
                #        - Decrement successor_job_info['unmet_dependencies'].
                #        - If successor_job_info['unmet_dependencies'] becomes 0:
                #            - Add the successor job to the pending heap:
                #              heapq.heappush(self.pending_jobs_heap,
                #                             (successor_job_info['priority'],
                #                              successor_job_info['submit_time'],
                #                              successor_id))
                #            - Print a message that the successor is now ready.
                pass # Replace with completion and dependency update logic

                completed_ids_this_cycle.append(job_id)
                print(f"  Job {job_id} ('{job['command']}') completed.")
                # Remove from running_jobs *after* processing successors
                # del self.running_jobs[job_id] # This should be part of the TODO above

    def _try_start_pending_jobs(self):
        """
        Internal helper to check the pending heap and start the highest-priority,
        dependency-free jobs that fit the available resources.
        """
        # --- Candidate TODO: Implement priority + dependency scheduling logic ---
        # While there are jobs in the heap AND the highest priority one *might* fit:
        #   - Peek at the highest priority job ID (self.pending_jobs_heap[0]).
        #   - Get the full job_info using the ID from self.all_jobs.
        #   - IMPORTANT: Double-check if job_info['unmet_dependencies'] is still 0.
        #     (It should be if it's in the heap, but good practice).
        #   - Check if it fits the self.free_slots.
        #   - If it fits AND dependencies are met (count is 0):
        #       - Extract the job ID (heapq.heappop).
        #       - Get the job_info again (in case it changed? Unlikely here but safer).
        #       - Update job state to RUNNING, record start_time, decrement free_slots.
        #       - Move job_info to running_jobs dict.
        #       - Continue (try to schedule the *next* highest priority job).
        #   - If it doesn't fit OR dependencies are not met:
        #       - Stop for this cycle (highest priority eligible job is blocked).
        # Note: A job might be popped from the heap but not run if resources become
        #       unavailable between peeking and popping (less likely in this single-threaded simulation).
        #       Consider how to handle this - maybe peek, check resources, then pop *if* runnable.
        pass # Replace with your implementation


    def run_scheduler_cycle(self):
        """
        Executes one cycle of the scheduler logic:
        1. Checks running jobs for completion (potentially freeing resources and resolving dependencies).
        2. Tries to start the highest-priority, dependency-free, pending jobs that fit.
        """
        print(f"\n--- Running Scheduler Cycle at {time.time():.2f} ---")
        self.show_status()

        # Check completions first to free up slots and resolve dependencies
        self._check_running_jobs()
        # Then try to start jobs based on priority, dependencies, and resources
        self._try_start_pending_jobs()

        print("--- Cycle Complete ---")
        self.show_status()

    def get_job_status(self, job_id: int) -> str | None:
        """
        Gets the current state of a specific job.
        """
        # --- Candidate TODO: Look up job state using self.all_jobs ---
        pass # Replace with your implementation
        return None # Placeholder

    def show_status(self):
        """
        Prints a summary of the scheduler's current state.
        """
        print(f"  Status: {self.free_slots}/{self.total_slots} slots free.")
        # --- Candidate TODO: Display pending jobs (heap count) and maybe waiting jobs ---
        # You could iterate self.all_jobs to find PENDING jobs with unmet_dependencies > 0
        pending_ready_count = len(self.pending_jobs_heap)
        print(f"  Pending Jobs (Ready in Heap): {pending_ready_count}")
        # Example: Count jobs waiting on dependencies
        waiting_deps_count = sum(1 for job in self.all_jobs.values() if job['state'] == PENDING and job['unmet_dependencies'] > 0)
        print(f"  Pending Jobs (Waiting Deps): {waiting_deps_count}")
        print(f"  Running Jobs: {len(self.running_jobs)}")
        print(f"  Completed Jobs: {len(self.completed_jobs)}")


# --- Example Usage (for candidate to test) ---
if __name__ == "__main__":
    print("Starting Dependency Scheduler Simulation")
    scheduler = DependencyScheduler(total_slots=10)

    # --- Candidate TODO: Add example usage with dependencies ---
    # Example:
    # jA = scheduler.submit_job("Task A", 4, priority=10) # Runs first
    # jB = scheduler.submit_job("Task B", 3, priority=5, depends_on=[jA]) # High prio, but waits for jA
    # jC = scheduler.submit_job("Task C", 5, priority=15, depends_on=[jA]) # Low prio, waits for jA
    # jD = scheduler.submit_job("Task D", 2, priority=8, depends_on=[jB, jC]) # Waits for B and C
    # jE = scheduler.submit_job("Task E", 3, priority=20) # Independent, low prio

    # scheduler.run_scheduler_cycle() # Should start jA
    # time.sleep(6) # Let jA finish
    # scheduler.run_scheduler_cycle() # Should complete jA, make jB, jC ready. Should start jB (higher prio)
    # time.sleep(6) # Let jB finish (assuming duration 5)
    # scheduler.run_scheduler_cycle() # Should complete jB. Should start jC (now highest prio ready) AND jE (if slots allow)
    # ... etc ...

    print("\nSimulation Complete.")
